# Validación: BH Accretion (EDR)
Este notebook estudia efectos EDR en discos de acreción estándar (toy models):
- Perfil de disco (Novikov–Thorne / Shakura–Sunyaev toy)
- Corrección EDR a la presión y torque
- Flujo de masa Ṁ modificado y luminosidad
- Comparación GR vs EDR
- Exportar resultados para análisis

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import simps
import json
plt.rcParams['figure.dpi'] = 120

## 1) Perfil de disco (toy Shakura–Sunyaev simplificado)
Superficie de densidad Σ(r) y energía liberada por unidad de área F(r) (modelo simplificado).

In [ ]:
def Sigma_disk(r, Sigma0=1e5, Rd=10.0):
    return Sigma0 * np.exp(-r/Rd)

def F_GR(r, M=1.0, Mdot=1e-3):
    # toy: F ~ Mdot / r^3 (dimensionalizado arbitrariamente)
    return Mdot / (r**3 + 1e-12)

## 2) Correcciones EDR al disco
Implementamos una corrección efectiva a la presión/torque que cambia F(r) → F(r) + δF_EDR(r).

In [ ]:
def deltaF_EDR(r, kflow=0.01, eta=0.02, scale=20.0):
    return kflow * eta * np.exp(-r/scale) / (r + 1e-6)

def F_total(r, Mdot=1e-3, kflow=0.01, eta=0.02):
    return F_GR(r, Mdot=Mdot) + deltaF_EDR(r, kflow, eta)

## 3) Flujo de masa y luminosidad integrada
L = 2π ∫ F(r) r dr  (integral radial)

In [ ]:
r = np.linspace(1.1, 100, 2000)
Fg = F_GR(r, M=1.0, Mdot=1e-3)
Fedr = F_total(r, Mdot=1e-3, kflow=0.02, eta=0.03)
L_gr = 2*np.pi*simps(Fg * r, r)
L_edr = 2*np.pi*simps(Fedr * r, r)
print({'L_gr': float(L_gr), 'L_edr': float(L_edr), 'deltaL_frac': float((L_edr-L_gr)/L_gr)})

## 4) Plots comparativos

In [ ]:
plt.figure(figsize=(7,4))
plt.loglog(r, Fg, label='F_GR')
plt.loglog(r, Fedr, label='F_GR + δF_EDR')
plt.xlabel('r [r_g]')
plt.ylabel('F(r)')
plt.legend(); plt.show()

## 5) Guardado de resultados para CI / paper

In [ ]:
res = {'L_gr': float(L_gr), 'L_edr': float(L_edr)}
import os
os.makedirs('results', exist_ok=True)
with open('results/BH_accretion_summary.json','w') as f:
    json.dump(res, f, indent=2)
print('Saved results/BH_accretion_summary.json')